# Proyecto2025_Riesgo_Operativo_Aerogeneradores_RioGrande_TDF

**Bloque:** Aprendizaje Automático  
**Autor:** Sabrina Villegas  
**Fecha:** 02 de Junio de 2025  
**Notebook:** 01 - Preprocesamiento de Datos  

---

## Objetivo

El objetivo de este notebook es realizar la carga, limpieza y preparación inicial de los datasets:

- Dataset **Operativo (SCADA)**: Registros de operación de aerogeneradores a intervalos de 10 minutos.
- Dataset **Meteorológico**: Datos climáticos diarios de Río Grande, Tierra del Fuego.

Las acciones principales incluyen:
- Conversión de variables de fecha/hora al formato adecuado.
- Detección y eliminación de duplicados.
- Inspección de datos faltantes (nulos) y su eliminacion.
- Unificación de ambos datasets por fecha, ajustando las frecuencias de muestreo.
- Guardado del dataset procesado para futuras etapas de análisis y modelado.

1. PREPROCESAMIENTO

In [309]:
# Importar librerías
import pandas as pd
import numpy as np

# Cargar datasets
path_operativo = '../data/raw/T1.csv'
path_meteorologico = '../data/raw/export.csv'


In [311]:
# Carga con ajuste de separador y encoding para meteorológico
df_operativo = pd.read_csv(path_operativo)
df_meteorologico = pd.read_csv(path_meteorologico)

In [313]:
# Visualizar las primeras filas
print("Dataset Operativo (SCADA):")
display(df_operativo.head())

print("Dataset Meteorológico:")
display(df_meteorologico.head())

Dataset Operativo (SCADA):


,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


Dataset Meteorológico:


,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2018-01-01 00:00:00,9.1,3.0,15.0,NaN,NaN,214.0,20.8,NaN,NaN,NaN
1,2018-01-02 00:00:00,7.3,5.0,10.0,NaN,NaN,146.0,15.2,NaN,NaN,NaN
2,2018-01-03 00:00:00,6.4,4.0,7.0,NaN,NaN,31.0,25.8,NaN,NaN,NaN
3,2018-01-04 00:00:00,8.7,6.0,12.0,NaN,NaN,351.0,19.5,NaN,NaN,NaN
4,2018-01-05 00:00:00,9.1,6.0,12.0,NaN,NaN,232.0,13.7,NaN,995.8,NaN


In [315]:
# Información de datasets
print("Información Dataset Operativo:")
df_operativo.info()

Información Dataset Operativo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50530 entries, 0 to 50529
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Date/Time                      50530 non-null  object 
 1   LV ActivePower (kW)            50530 non-null  float64
 2   Wind Speed (m/s)               50530 non-null  float64
 3   Theoretical_Power_Curve (KWh)  50530 non-null  float64
 4   Wind Direction (°)             50530 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.9+ MB


In [317]:
print("Información Dataset Meteorológico:")
df_meteorologico.info()

Información Dataset Meteorológico:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    352 non-null    object 
 1   tavg    352 non-null    float64
 2   tmin    332 non-null    float64
 3   tmax    326 non-null    float64
 4   prcp    120 non-null    float64
 5   snow    6 non-null      float64
 6   wdir    319 non-null    float64
 7   wspd    324 non-null    float64
 8   wpgt    0 non-null      float64
 9   pres    78 non-null     float64
 10  tsun    0 non-null      float64
dtypes: float64(10), object(1)
memory usage: 30.4+ KB


In [319]:
# Conversión de fechas
df_operativo['Date/Time'] = pd.to_datetime(df_operativo['Date/Time'], dayfirst=True)
df_meteorologico['date'] = pd.to_datetime(df_meteorologico['date'])

In [321]:
# Chequeo de duplicados, si existen se eliminan 
print(f"Duplicados en Operativo: {df_operativo.duplicated().sum()}")
print(f"Duplicados en Meteorológico: {df_meteorologico.duplicated().sum()}")


Duplicados en Operativo: 0
Duplicados en Meteorológico: 0


In [323]:
# LIMPIEZA inteligente de valores nulos
# Eliminar columnas con más del 40% de valores nulos
threshold = 0.4
df_operativo = df_operativo.dropna(axis=1, thresh=int((1-threshold) * len(df_operativo)))
df_meteorologico = df_meteorologico.dropna(axis=1, thresh=int((1-threshold) * len(df_meteorologico)))

In [325]:
# Eliminar filas con nulos remanentes
df_operativo = df_operativo.dropna()
df_meteorologico = df_meteorologico.dropna()

In [327]:
# Merge de datasets por fecha
df_operativo_daily = df_operativo.resample('D', on='Date/Time').mean().reset_index()

In [329]:
df_merged = pd.merge(df_operativo_daily, df_meteorologico, left_on='Date/Time', right_on='date')

In [331]:
# Verificar el dataset unificado
print("Dataset Unificado:")
display(df_merged.head())

Dataset Unificado:


,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°),date,tavg,tmin,tmax,wdir,wspd
0,2018-01-01,1535.204483,8.022975,1622.990048,229.735242,2018-01-01,9.1,3.0,15.0,214.0,20.8
1,2018-01-02,3182.564250,12.000548,3314.693471,197.859789,2018-01-02,7.3,5.0,10.0,146.0,15.2
2,2018-01-03,1017.793340,6.650189,1072.956361,192.476045,2018-01-03,6.4,4.0,7.0,31.0,25.8
3,2018-01-04,834.486869,6.232531,838.463088,121.299142,2018-01-04,8.7,6.0,12.0,351.0,19.5
4,2018-01-05,123.717345,3.121434,130.342291,154.906666,2018-01-05,9.1,6.0,12.0,232.0,13.7


In [333]:
# Guardar dataset procesado
df_merged.to_csv('../data/processed/merged_dataset.csv', index=False)
print("Dataset unificado guardado correctamente en ../data/processed/merged_dataset.csv")

Dataset unificado guardado correctamente en ../data/processed/merged_dataset.csv
